In [1]:
import time
import datetime
import requests
import pandas as pd
# Treverse file
import os

In [2]:
# Convert unix time stmap
def convert_time(ts):
    time = datetime.datetime.fromtimestamp(ts)
    return str(time)
#=(C2+8*3600)/86400+70*365+19

In [3]:
# Drop null values
def drop_empty(dataframe):
    for i in range(0,len(dataframe)):
        if dataframe.loc[i,'close'] == 0:
            dataframe = dataframe.drop([i])
        else:
            break
    return dataframe

In [4]:
# Waiting status display
def wait(sleepTime):
    print("Waiting...restart after:")
    for i in range(0, sleepTime):
        print("    " + str(sleepTime) +" seconds")
        time.sleep(1)
        sleepTime -= 1

In [5]:
# Get price data via connecting API
def get_price_data(apiKey, url_data, exchange, fsym, tsym, limit, time_now, rounds, end):
    price_data = pd.DataFrame(columns=('time', 'close','high','low','open','volumefrom','volumeto'))
    start_time = time_now
    payload_data = {
        "api_key": apiKey,
        "e" : exchange,
        "fsym": fsym,
        "tsym": tsym,
        "limit": limit,
        "toTs": start_time
        }

    for i in range(0,rounds):
        while(True):
            try: 
                print("Getting data...")
                response = requests.get(url_data, params=payload_data, timeout=20)
                response.raise_for_status()
                result = response.json()
                df = pd.DataFrame(result['Data'])
                price_data = df.append(price_data)
                start_time = df.head(1).time.values[0] + 60
                break
            except:
                print("Connection refused by the server..")
                wait(5)
                print("Continue...")
                continue
            else:
                print("Success")
        
#  end_ts = price_data.tail(1).time.values[0]
    file_name = "D:/Data/" + exchange + "_hourly_" + fsym + "_" + tsym + "_" + str(end) + ".csv"
    price_data_reindex = price_data.reset_index()
    price_data_dropnull = drop_empty(price_data_reindex)
    
    price_data_dropnull.to_csv(file_name)
    print("Successfully saved as file: " + str(file_name))

In [6]:
# Get pair information via connecting API
def get_histo_data(apiKey, exchange, fsym, tsym, last_file, end):
    # Start time
    time_now = int(time.time())

    # Config data volumn parm
    payload_volumn = {
        "api_key": apiKey,
        "e" : exchange
        }

    # Get data total volumn
    # Use /exchanges to get exchanges data
    url_vol = "https://min-api.cryptocompare.com/data/v4/all/exchanges"
    
    # Url to connect with API
    url_data = "https://min-api.cryptocompare.com/data/histohour"

    # Request data
    while(True):
        try:
            result_volumn = requests.get(url_vol, params=payload_volumn, timeout=20).json()
            break
        except:
            print("Connection refused by the server..")
            wait(5)
            print("Continue...")
            continue

    # Get start and end timestamp
    try:
        pair_start_at = result_volumn['Data']['exchanges'][exchange]['pairs'][fsym]['tsyms'][tsym]['histo_minute_start_ts']
        pair_end_at = result_volumn['Data']['exchanges'][exchange]['pairs'][fsym]['tsyms'][tsym]['histo_minute_end_ts']
    except BaseException:
        # If any of parameters is wrong, terminated process
        print("Parameter Error! process terminated...")
        return

    # Calculate data volumn & rounds of the loop based on last file
    rounds = 1
    limit = 2000
    print("Start capturing data: "+ fsym + " to "+ tsym + " in " + exchange)
    try:
        pre_data = pd.read_csv(last_file)
        # If last file is existed
        print(last_file + " Found. ")
        print("Continue capture data...")
        print("----------------------------------------------" )
        pair_start_at = pre_data.tail(1).time.values[0]
        print("Last data end at:  " + convert_time(pair_start_at))
        print("Start capture at:  " + convert_time(time_now))
        # Total volumn of the pair
        data_total_volumn = int(round((time_now - pair_start_at) / 3600)) - 1
        if data_total_volumn >= 2000:
            rounds = round(data_total_volumn / 2000)
        else:
            limit = data_total_volumn
        print("Need to be added:  " + str(data_total_volumn))
        # Get price data
        print("----------------------------------------------" )
        if data_total_volumn == 0:
            print("No data to update")
            print("Terminated...")
        else:
            print("loading...")
            get_price_data(apiKey, url_data,exchange, fsym, tsym, limit, time_now, rounds, end)
    except FileNotFoundError:
        # If last file is not existed
        if len(last_file) == 0:
            # If there is no previous file
            print("History file Not Found.")
        else:
            # If previous file is not existed
            print(last_file + " Not Found.")
        print("Create new file...")
        print("----------------------------------------------" )
        print("History data start at:  " + convert_time(pair_start_at))
        print("Start capture at:       " + convert_time(time_now))
         # Total volumn of the pair
        data_total_volumn = int(round((time_now - pair_start_at) / 3600))
        if data_total_volumn >= 2000:
            rounds = round(data_total_volumn / 2000)
        else:
            limit = data_total_volumn
        print("Total data volumn:      " + str(data_total_volumn))
        print("----------------------------------------------" )
        print("loading...")
        # Get price data
        get_price_data(apiKey, url_data, exchange, fsym, tsym, limit, time_now, rounds, end)

In [8]:
# Load the objective exchange(s) info
# You must get the exchange data first
def load_exchange_addres_sbook(file_name):
    return pd.read_csv('all_ex.csv')

In [9]:
def find_exist_file(filePath):
    return os.listdir(filePath)

In [10]:
def is_file_exits(file_name, all_files):
    return file_name in all_files

In [11]:
def get_multi_pairs(apiKey, exchanges, all_files):
    exist_file_volumn = 0
    add_file_volumn = 0
    last_file = ""
    for i in range(0, len(exchanges)):
        # Get exchange and pair info
        exchange_name = exchanges.loc[i, 'exchange']
        fsym = exchanges.loc[i, 'fsym']
        tsym = exchanges.loc[i, 'tsym']
        end = exchanges.loc[i, 'end']
        # Generate standard data file name
        file_name =  exchange_name + "_hourly_" + fsym + "_" + tsym + "_" + str(end) + ".csv"
        if is_file_exits(file_name, all_files):
            print(file_name + " is existed jump over")
            exist_file_volumn += 1
            continue
        else:
            get_histo_data(apiKey, exchange_name, fsym, tsym, last_file, end)
            print("Last time file volumn: " + str(exist_file_volumn))
            add_file_volumn += 1
            print("File added: " + str(add_file_volumn))
            print("\n")

In [ ]:
# Main function1
# Get one specific crypto pair history data

#=====================(Param Sample)=========================
# Use your api key   
apiKey = "1bd6b034f5c37235fe86bd57125805ae1164b1f50aea15648073d27a3627b00e"

# Exchange name:  

#exchange = "HuobiPro"  (Sample)

# Pair 

#fsym = "BSV"         (Sample)
#tsym = "USDT"      (Sample)

# Current unix time
end = int(time.time())
#==========================================================

#========================(Optional)============================
# Last file 
# The data file captured last time, 
# If you want to continue capture data, 
# Please fill in the last_file with previous file name
# Otherwise 'last_file' set with empty String as default

    last_file = ""
#==========================================================

get_histo_data(exchange, fsym, tsym, end, apiKey)

In [ ]:
# Main function2
# Get multiple crypto pairs history data

#=====================(Param Sample)=========================
# Use your api key   
apiKey = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Param 'exchanges' represent all exchanges you wish to capture, 
#     please use the sample dataset 'all_ex.csv',
#
#     or you can setup another exchanges list with the same format
#     FORMAT is very important! Please check file 'all_ex.csv'
exchanges = load_exchange_addres_sbook('all_ex.csv').reset_index()

# Param 'all_files' represent all previous data captured.
#     Use function 'find_exist_file()' with data storage location to get all previous file,
#     witch could help you easy to continue your previous work.
#
#     Recommand to use this location, please create this folder before start
all_files = find_exist_file('D:\\Data') 

#==========================================================

get_multi_pairs(apiKey, exchanges, all_files)